# Executive Overview Section

In [3]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

# Load Data
df = pd.read_csv("C:/Users/harsh/OneDrive/Desktop/Unified Mentor/WA_Fn-UseC_-HR-Employee-Attrition.csv")

# Clean Data: Drop constant columns
df.drop(columns=['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber'], inplace=True)

# Compute KPIs
total_employees = df.shape[0]
total_attrition = df[df['Attrition'] == 'Yes'].shape[0]
attrition_rate = (total_attrition / total_employees) * 100
avg_income = df['MonthlyIncome'].mean()
avg_tenure = df['YearsAtCompany'].mean()

# Initialize Dash App with Bootstrap Theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'IBM HR Analytics Dashboard'

# KPI Cards
def kpi_card(title, value, color='primary'):
    return dbc.Card([
        dbc.CardBody([
            html.H5(title, className='card-title'),
            html.H3(value, className='card-text')
        ])
    ], color=color, inverse=True, className='m-2')

# Layout
app.layout = dbc.Container([
    html.H1("IBM HR Analytics Dashboard", className="text-center my-4"),

    dbc.Row([
        dbc.Col(kpi_card("Total Employees", f"{total_employees}"), width=2),
        dbc.Col(kpi_card("Attrition Count", f"{total_attrition}", color='danger'), width=2),
        dbc.Col(kpi_card("Attrition Rate", f"{attrition_rate:.2f}%", color='warning'), width=2),
        dbc.Col(kpi_card("Avg Monthly Income", f"${avg_income:,.0f}"), width=3),
        dbc.Col(kpi_card("Avg Tenure (Years)", f"{avg_tenure:.1f}"), width=3),
    ], justify="center")

], fluid=True)

# Run app
if __name__ == '__main__':
    app.run(debug=True, port=8051)

In [4]:
html.Div(id="kpi-cards", className="mb-4")


Div(id='kpi-cards', className='mb-4')

In [5]:
dcc.Dropdown(
    id='department-filter', 
    options=[{'label': 'HR', 'value': 'HR'}, {'label': 'Sales', 'value': 'Sales'}],
    value='HR'
),
dcc.Dropdown(
    id='gender-filter',
    options=[{'label': 'Male', 'value': 'Male'}, {'label': 'Female', 'value': 'Female'}],
    value='Male'
)


Dropdown(options=[{'label': 'Male', 'value': 'Male'}, {'label': 'Female', 'value': 'Female'}], value='Male', id='gender-filter')

# Interactive Filter Pannel

In [6]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

# Load dataset
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")

# Drop unnecessary columns
df.drop(columns=['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber'], inplace=True)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'IBM HR Analytics Dashboard'

# KPI card layout function
def kpi_card(title, value, color='primary'):
    return dbc.Card(
        dbc.CardBody([
            html.H5(title, className='card-title'),
            html.H3(value, className='card-text')
        ]),
        color=color, inverse=True, className='m-2'
    )

# Layout with top-row filters
app.layout = dbc.Container([
    html.H2("IBM HR Attrition Dashboard", className="text-center my-4"),

    # Top-row filters
    dbc.Row([
        dbc.Col([
            html.Label("Department"),
            dcc.Dropdown(
                options=[{'label': i, 'value': i} for i in df['Department'].unique()],
                id='filter-department',
                placeholder="All Departments", multi=True
            )
        ], width=3),

        dbc.Col([
            html.Label("Gender"),
            dcc.Dropdown(
                options=[{'label': i, 'value': i} for i in df['Gender'].unique()],
                id='filter-gender',
                placeholder="All Genders", multi=True
            )
        ], width=2),

        dbc.Col([
            html.Label("Age Range"),
            dcc.RangeSlider(
                id='filter-age',
                min=df['Age'].min(),
                max=df['Age'].max(),
                step=1,
                marks={age: str(age) for age in range(df['Age'].min(), df['Age'].max()+1, 5)},
                value=[df['Age'].min(), df['Age'].max()]
            )
        ], width=5),

        dbc.Col([
            html.Label("Education Field"),
            dcc.Dropdown(
                options=[{'label': i, 'value': i} for i in df['EducationField'].unique()],
                id='filter-edu',
                placeholder="All Fields", multi=True
            )
        ], width=2),
    ], className='mb-3'),

    dbc.Row([
        dbc.Col([
            html.Label("Job Role"),
            dcc.Dropdown(
                options=[{'label': i, 'value': i} for i in df['JobRole'].unique()],
                id='filter-job',
                placeholder="All Roles", multi=True
            )
        ], width=3),

        dbc.Col([
            html.Label("Marital Status"),
            dcc.Dropdown(
                options=[{'label': i, 'value': i} for i in df['MaritalStatus'].unique()],
                id='filter-marital',
                placeholder="All Statuses", multi=True
            )
        ], width=2),

        dbc.Col([
            html.Label("Work-Life Balance"),
            dcc.Dropdown(
                options=[{'label': f'Level {i}', 'value': i} for i in sorted(df['WorkLifeBalance'].unique())],
                id='filter-wlb',
                placeholder="All Levels", multi=True
            )
        ], width=2),

        dbc.Col([
            html.Label("Years at Company"),
            dcc.RangeSlider(
                id='filter-tenure',
                min=df['YearsAtCompany'].min(),
                max=df['YearsAtCompany'].max(),
                step=1,
                marks={i: str(i) for i in range(0, df['YearsAtCompany'].max()+1, 5)},
                value=[df['YearsAtCompany'].min(), df['YearsAtCompany'].max()]
            )
        ], width=5),
    ], className='mb-4'),

    # KPI cards row
    dbc.Row(id='kpi-row', justify='center'),

], fluid=True)

# Callback to update KPIs based on all filters
@app.callback(
    Output('kpi-row', 'children'),
    Input('filter-department', 'value'),
    Input('filter-gender', 'value'),
    Input('filter-age', 'value'),
    Input('filter-edu', 'value'),
    Input('filter-job', 'value'),
    Input('filter-marital', 'value'),
    Input('filter-wlb', 'value'),
    Input('filter-tenure', 'value')
)
def update_kpis(department, gender, age, edu, job, marital, wlb, tenure):
    dff = df.copy()
    if department:
        dff = dff[dff['Department'].isin(department)]
    if gender:
        dff = dff[dff['Gender'].isin(gender)]
    if age:
        dff = dff[(dff['Age'] >= age[0]) & (dff['Age'] <= age[1])]
    if edu:
        dff = dff[dff['EducationField'].isin(edu)]
    if job:
        dff = dff[dff['JobRole'].isin(job)]
    if marital:
        dff = dff[dff['MaritalStatus'].isin(marital)]
    if wlb:
        dff = dff[dff['WorkLifeBalance'].isin(wlb)]
    if tenure:
        dff = dff[(dff['YearsAtCompany'] >= tenure[0]) & (dff['YearsAtCompany'] <= tenure[1])]

    # Calculated KPIs
    total_employees = dff.shape[0]
    total_attrition = dff[dff['Attrition'] == 'Yes'].shape[0]
    attrition_rate = (total_attrition / total_employees) * 100 if total_employees > 0 else 0
    avg_income = dff['MonthlyIncome'].mean()
    avg_tenure = dff['YearsAtCompany'].mean()

    return [
        dbc.Col(kpi_card("Total Employees", f"{total_employees}"), width=2),
        dbc.Col(kpi_card("Attrition Count", f"{total_attrition}", color='danger'), width=2),
        dbc.Col(kpi_card("Attrition Rate", f"{attrition_rate:.2f}%", color='warning'), width=2),
        dbc.Col(kpi_card("Avg Monthly Income", f"${avg_income:,.0f}"), width=3),
        dbc.Col(kpi_card("Avg Tenure (Years)", f"{avg_tenure:.1f}"), width=3),
    ]

# Run the Dash app
if __name__ == "__main__":
    app.run(debug=True, port=8052)


# Attrition, Demographic & Performance

In [7]:
df['JoinYear'] = 2025 - df['YearsAtCompany']

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'Step 3 - Visualizations'

app.layout = dbc.Container([
    html.H2("Step 3: Visualizations Dashboard", className="text-center mt-4 mb-3"),

    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='📊 Attrition Analysis', value='tab-1'),
        dcc.Tab(label='👥 Demographic Insights', value='tab-2'),
        dcc.Tab(label='📈 Performance & Satisfaction', value='tab-3'),
    ]),

    html.Div(id='tabs-content')
], fluid=True)

@app.callback(Output('tabs-content', 'children'), Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return dbc.Container([
            dbc.Row([
                dbc.Col(dcc.Graph(figure=px.bar(
                    df[df['Attrition'] == 'Yes'].groupby(['JobRole', 'Gender']).size().reset_index(name='Count'),
                    x='JobRole', y='Count', color='Gender',
                    title='Attrition by Job Role and Gender'
                )), width=6),

                dbc.Col(dcc.Graph(figure=px.bar(
                    df[df['Attrition'] == 'Yes'].groupby(['Department', 'EducationField']).size().reset_index(name='Count'),
                    x='Department', y='Count', color='EducationField',
                    title='Attrition by Department and Education Field', barmode='stack'
                )), width=6),
            ]),

            dbc.Row([
                dbc.Col(dcc.Graph(figure=px.density_heatmap(
                    df[df['Attrition'] == 'Yes'],
                    x='WorkLifeBalance', y='JobSatisfaction',
                    title='Heatmap: Attrition vs Work-Life Balance & Job Satisfaction'
                )), width=12)
            ])
        ])

    elif tab == 'tab-2':
        return dbc.Container([
            dbc.Row([
                dbc.Col(dcc.Graph(figure=px.pie(
                    df, names='Gender', title='Gender Distribution'
                )), width=4),

                dbc.Col(dcc.Graph(figure=px.histogram(
                    df, x='Age', nbins=20, title='Age Distribution'
                )), width=4),

                dbc.Col(dcc.Graph(figure=px.box(
                    df, x='Gender', y='MonthlyIncome', title='Monthly Income by Gender'
                )), width=4),
            ])
        ])

    elif tab == 'tab-3':
        radar_data = go.Figure(data=go.Scatterpolar(
            r=[
                df['EnvironmentSatisfaction'].mean(),
                df['JobSatisfaction'].mean(),
                df['RelationshipSatisfaction'].mean(),
                df['WorkLifeBalance'].mean()
            ],
            theta=['Environment', 'Job', 'Relationship', 'Work-Life'],
            fill='toself'
        ))
        radar_data.update_layout(
            polar=dict(radialaxis=dict(visible=True)),
            title='Radar Chart: Satisfaction Scores'
        )

        return dbc.Container([
            dbc.Row([
                dbc.Col(dcc.Graph(figure=px.scatter(
                    df, x='YearsAtCompany', y='PerformanceRating', color='JobRole',
                    title='Years at Company vs Performance Rating'
                )), width=6),

                dbc.Col(dcc.Graph(figure=px.bar(
                    df.groupby(['EnvironmentSatisfaction', 'JobInvolvement']).size().reset_index(name='Count'),
                    x='EnvironmentSatisfaction', y='Count', color='JobInvolvement',
                    title='Environment Satisfaction & Job Involvement'
                )), width=6),
            ]),

            dbc.Row([
                dbc.Col(dcc.Graph(figure=radar_data), width=12)
            ])
        ])

if __name__ == '__main__':
    app.run(debug=True, port=8053)


# Time Based Trends and Insights

In [8]:
df['JoinYear'] = 2025 - df['YearsAtCompany']

# Preprocess for line trend
attrition_trend = df[df['Attrition'] == 'Yes'].groupby('JoinYear').size().reset_index(name='Attritions')

# Initialize App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
app.title = 'Step 4 & 5 - Trends and Insights'

app.layout = dbc.Container([
    html.H2("Step 4: Time-Based Trends", className="text-center mt-4"),

    dbc.Row([
        dbc.Col(dcc.Graph(figure=px.line(
            attrition_trend, x='JoinYear', y='Attritions',
            markers=True, title='Attrition Trend Over Join Year'
        )), width=12)
    ]),

    html.Hr(className="my-4"),

    html.H2("Step 5: Insights & Recommendations", className="text-center"),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Top 3 Factors Influencing Attrition", className="bg-primary text-white"),
                dbc.CardBody([
                    html.Ul([
                        html.Li("Job Role & Department mismatch"),
                        html.Li("Low Work-Life Balance & Job Satisfaction"),
                        html.Li("High Overtime & Low Income")
                    ])
                ])
            ], className="mb-4"),

            dbc.Card([
                dbc.CardHeader("Departments At Risk", className="bg-warning"),
                dbc.CardBody([
                    html.Ul([
                        html.Li("Sales Department"),
                        html.Li("Research & Development")
                    ])
                ])
            ], className="mb-4"),

            dbc.Card([
                dbc.CardHeader("Strategic HR Recommendations", className="bg-success text-white"),
                dbc.CardBody([
                    html.Ul([
                        html.Li("Improve work-life balance with flexible hours."),
                        html.Li("Increase satisfaction in low-engagement roles."),
                        html.Li("Provide tailored incentives for high-risk groups.")
                    ])
                ])
            ])
        ], width=12)
    ])
], fluid=True)

if __name__ == '__main__':
    app.run(debug=True, port=8054)


In [9]:
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

# Simulate JoinYear
df['JoinYear'] = 2025 - df['YearsAtCompany']

# Dash App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'IBM HR Dashboard - Styling & Search'

# KPI Cards Function
def create_kpi_cards(filtered_df):
    return dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("Total Employees"),
            dbc.CardBody(html.H4(f"{len(filtered_df)}", className="card-title"))
        ], color="primary", inverse=True), width=4),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Attrition Count"),
            dbc.CardBody(html.H4(f"{(filtered_df['Attrition'] == 'Yes').sum()}", className="card-title"))
        ], color="danger", inverse=True), width=4),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Average Monthly Income"),
            dbc.CardBody(html.H4(f"${filtered_df['MonthlyIncome'].mean():.2f}", className="card-title"))
        ], color="success", inverse=True), width=4),
    ])

# Layout
app.layout = dbc.Container([
    html.H2("IBM HR Dashboard - Professional Polish", className="text-center my-4"),

    dbc.Row([
        dbc.Col([
            dbc.Input(id='search-input', placeholder='Search by Job Role or Department...', type='text'),
            html.Br(),
            dbc.Button("Search", id='search-button', color='info')
        ], width=8),

        dbc.Col([
            html.Div([
                html.Span("ℹ️", id="tooltip-target", style={"cursor": "pointer", "fontSize": "24px"}),
                dbc.Tooltip(
                    "Search for a specific Job Role or Department to filter employee insights.",
                    target="tooltip-target",
                    placement="right"
                )
            ])
        ], width=1)
    ], className='mb-4'),

    dbc.Spinner(children=[html.Div(id='kpi-cards')], color='primary'),

    html.Div(id='description-text', className='mt-3 text-secondary')
], fluid=True)

# Callbacks
@app.callback(
    Output('kpi-cards', 'children'),
    Output('description-text', 'children'),
    Input('search-button', 'n_clicks'),
    State('search-input', 'value')
)
def update_kpis(n_clicks, search_value):
    if not search_value:
        filtered = df
    else:
        filtered = df[
            df['JobRole'].str.contains(search_value, case=False, na=False) |
            df['Department'].str.contains(search_value, case=False, na=False)
        ]

    kpi_cards = create_kpi_cards(filtered)
    description = f"Filtered by: '{search_value}' — Showing insights for {len(filtered)} employee(s)." if search_value else "Showing overall KPIs for all employees."
    return kpi_cards, description

# Run App
if __name__ == '__main__':
    app.run(debug=True, port=8056)
